### Data Preprocessing ###

In [5]:
import re
import jieba
import jieba.analyse
import keras
import pandas as pd
import numpy as np
from pythonbq import pythonbq
from keras.models import load_model

class NewsClassifier():

    def __init__(self, model_path):
        self.model = load_model(model_path)
        
    # 取字詞資料集並匯出csv
    def getWord_index_csv(self):
        my_project_id='maximal-cabinet-254805'
        my_dataset_id='news_keywords'
        credentials_path = 'C:/Users/GameToGo/MyCredentials/My First Project-7226ff9d97d3.json'
        
        myProject=pythonbq(
          bq_key_path = credentials_path,
          project_id = my_project_id
        )
        SQL_CODE="""
            SELECT * FROM `maximal-cabinet-254805.news_keywords.keywords` 
            order by id
        """

        output=myProject.query(sql=SQL_CODE)
        output.to_csv('word_index.csv', index = False)

    # 讀取字詞集csv
    def getWord_index(self):
        word_df = pd.read_csv('./word_index.csv')

        word_index = {}
        for w in word_df.values:
            word_index[w[1]] = w[0]

        return word_index

    # 關鍵字提取(基于TF-IDF算法)
    def getTxtArray(self, txt, K, word_index):
        pattern = re.compile(r'[\d+\u4e00-\u9fa5]+') # 取中文加數字
        txt_list = pattern.findall(txt)
        txt2 = ''.join(txt_list)

        keywords = jieba.analyse.extract_tags(
                            sentence=txt2, topK=K, withWeight=True, allowPOS=('n','nr','ns'))
        words = []
        txt_index = []
        for item in keywords:
            key = item[0]
            words.append(key)
            if key in word_index.keys():
                txt_index.append(word_index[key])   
            else:
                txt_index.append(0) 
        
        return words, [txt_index]
    
    # 匯入model與預測
    def getPredictAns(self, txt):
        s = ["政治", "運動", "財經", "社會", "房產", "國際", "娛樂"]
        
        word_index = self.getWord_index()
        words, txt_index = self.getTxtArray(txt, 20, word_index)
        # 關鍵字不滿20補0
        txt_seq = keras.preprocessing.sequence.pad_sequences(txt_index, maxlen = 20)
        
        pre = self.model.predict(txt_seq)
        ans = pre.argmax(axis=1)[0]
        return words, s[ans]

In [6]:
txt = "高雄市長韓國瑜，在歷經2020總統大選大敗後，也回歸高雄市政工作。不過緊接著而來他所要面對的，則有由weCare發起的罷免韓國瑜活動，至今已經收到超過21萬份的連署書，距離法定門檻只剩下一萬多份的距離。對此韓國瑜回歸市政以來，不僅行事作風低調許多，更頻頻打出政績牌。今天（11日）韓國瑜更召開記者會對外宣布，將提供2萬美金以及來回機票，給大專院校學生及公教警消人員出國進修。韓國瑜今天上午召開會，宣布啟動「滿天星計畫」，今年先動用教育基金，補助100位在地大專院校學生出國研修，每人1年2萬美金，明年則將送軍公教警消出國，幫助高雄年輕人開拓視野。而這項計畫一共分成兩部分，第一階段高雄市政府將針對設籍高雄5年以上、就讀在地18所大專院校的學生，共100個名額，1年補助每人2萬美金，以及來回飛機票，協助他們出國研修，其中也將保留一定數量弱勢保障名額，讓所有孩子均有機會一圓出國夢。而第二階段則針對軍公教警消，預計明年挑選優秀公務人員，同樣提供每人1年2萬美金補助到國外姊妹市相關學校進修，按照計畫草案，申請年齡暫訂簡任級55歲以下、薦任50歲、委任45歲以下，希望為市府施政引入更多國外新進理念。韓國瑜表示，台灣未來最重要的就是人才。「市府率先推出滿天星計畫，預計青年學子出國回來，語言能力、解決生活困難能力、與世界交朋友能力以及膽量都會增加。」韓國瑜說，「儘管市府財政困難，但人才培育應列為頭等大事，期待這項計畫若干年後能開花結果。」不過罷韓連署仍如火如荼進行當中，今天最新的連署書數量，更已經一舉突破21萬，距離法定門檻的22萬8千份也僅剩下一步之遙。儘管罷韓團體表示會盡量累積到30萬份再做交件的動作，不過距離選舉結束至今不到一個月時間，仍有超過20萬人支持霸韓行動，也顯示韓國瑜若是在市政上再無起色，恐怕真的會面臨遭到罷免的命運。"

classifier = NewsClassifier('./model/7news_lstm_acc89.h5')
words, ans = NewsClassifier.getPredictAns(classifier, txt)
print(ans)
print(words)

財經
['韓國瑜', '計畫', '美金', '距離', '公教', '高雄市', '市府', '院校', '門檻', '機票', '學生', '人員', '階段', '名額', '數量', '高雄', '市政', '法定', '能力', '人才']
